# Basic demonstration of LlamaIndex integration functionality with the Unity Catalog AI Toolkit SDK

To get started with this, you will need an OpenAI API Key. 

Once you have acquired your key, set it to the environment variable `OPENAI_API_KEY` after storing it in the `Databricks Secrets` API (accessible via dbutils or the databricks sdk workspace client).

In [0]:
%pip install -Uqqq unitycatalog-llamaindex[databricks] openai llama_index mlflow

%restart_python

## Setting your API Key

Don't forget to remove the key after you're done running cell 4!

In [0]:
import base64
import os

from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()

secret_scope = "ben_wilson"  # Change me!

# Run this if you don't have the API key set to your secrets scope yet

# if secret_scope not in [scope.name for scope in workspace_client.secrets.list_scopes()]:
#     workspace_client.secrets.create_scope(secret_scope)

# my_secret = "<your API key, temporarily>"

# workspace_client.secrets.put_secret(scope=secret_scope, key="openai_api_key", string_value=my_secret)

## Fetch the key and set it to the environment variable key that the OpenAI SDK needs

In [0]:
os.environ["OPENAI_API_KEY"] = base64.b64decode(
    workspace_client.secrets.get_secret(scope=secret_scope, key="openai_api_key").value
).decode()

In [0]:
assert "OPENAI_API_KEY" in os.environ, (
    "Please set the OPENAI_API_KEY environment variable to your OpenAI API key"
)

## Import the UC client for Databricks UC. 
This will allow for function creation through either the `create_function` API (requires the defined `sql_body` statement) or the `create_python_function` (requires a type-hint-applied and docstring commented python callable). 

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from unitycatalog.ai.llama_index.toolkit import UCFunctionToolkit

## Set the UC Catalog and Schema 

You must set both of these that you will be using to store and execute your function(s). If these do not exist, ensure that you create them first. 

In [0]:
CATALOG = "ben_wilson"  # Change me!
SCHEMA = "uc_func"  # Change me if you want

In [0]:
client = DatabricksFunctionClient()

### Define a Callable
The requirements for the callable:

**typing**

Types **must** be supplied for both the arguments and the return type. Function signatures that do not have these defined will raise a `ValueError`.

The following types are not allowed:
`Union`
`Any`

Additional caveats:
Collections **must** supply typing of the interior components. For instance, ``typing.Dict`` is not allowed, but ``typing.Dict[str, str]`` will work correctly. 

**doc strings**

The doc string **must** be in the Google Docstring format.
Args and Returns comments are optional, but the function description **is required**. 

In [0]:
def fetch_cold_weather(location: str) -> str:
    """
    Fetches the current weather in celsius for a given location.

    Args:
        location (str): The location to fetch the weather for.

    Returns:
        str: The current weather in celsius for the given location.
    """

    return "1.9 C"

In [0]:
client.create_python_function(func=fetch_cold_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(catalog_name='ben_wilson', comment='Fetches the current weather in celsius for a given location.', created_at=1729720429548, created_by='benjamin.wilson@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language='Python', external_name=None, full_data_type='STRING', full_name='ben_wilson.uc_func.fetch_cold_weather', function_id='40ee2d86-ac4a-4b7e-b3ec-03f03f80ccb5', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='The location to fetch the weather for.', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"location","type":"string","nullable":true,"metadata":{"comment":"The location to fetch the weather for."}}', type_precision=0, type_scale=0)]), is_deterministic=False, is_null_call=None, metastore_id='19a85dee-54bc-43a2-87ab-023d0ec16013

In [0]:
# Create a tool instance to use with LlamaIndex

toolkit = UCFunctionToolkit(
    function_names=[f"{CATALOG}.{SCHEMA}.fetch_cold_weather"], client=client
)

## Enable tracing in MLflow
Auto-enabling tracing allows us to see the calls made by LlamaIndex

In [0]:
import mlflow

mlflow.llama_index.autolog()

## Submit the question

In the question request, submit the defined tools from the `UCFunctionToolkit` instance functions that have been fetched. 

In [0]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

llm = OpenAI()

agent = ReActAgent.from_tools(toolkit.tools, llm=llm, verbose=True)

agent.chat("Please call a python execution tool to fetch the weather in Bogota Columbia.")

> Running step 727d788f-557a-4d5e-9c76-ac036145121b. Step input: Please call a python execution tool to fetch the weather in Bogota Columbia.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: ben_wilson__uc_func__fetch_cold_weather
Action Input: {'properties': AttributedDict([('location', 'Bogota, Colombia')])}
Observation: {"format": "SCALAR", "value": "1.9 C"}
> Running step d44063ae-b5c3-4360-ac26-0926e2c87047. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current weather in Bogota, Colombia is 1.9 degrees Celsius.


AgentChatResponse(response='The current weather in Bogota, Colombia is 1.9 degrees Celsius.', sources=[ToolOutput(content='{"format": "SCALAR", "value": "1.9 C"}', tool_name='ben_wilson__uc_func__fetch_cold_weather', raw_input={'args': (), 'kwargs': {'properties': AttributedDict([('location', 'Bogota, Colombia')])}}, raw_output='{"format": "SCALAR", "value": "1.9 C"}', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

Trace(request_id=tr-1281691e49bb4cf0b9b428efc3f359c9)